# [CMI-SleepState-Detection](https://www.kaggle.com/competitions/child-mind-institute-detect-sleep-states/overview)
## Child Mind Institute - Detect Sleep States
### Detect sleep onset and wake from wrist-worn accelerometer data
_______________________________________________________________________ 
# Author Details:
- Name: Najeeb Haider Zaidi
- Email: zaidi.nh@gmail.com
- Profiles: [Github](https://github.com/snajeebz)  [LinkedIn](https://www.linkedin.com/in/najeebz) [Kaggle](https://www.kaggle.com/najeebz)
- License: Private, Unlicensed, All the files in this repository under any branch are Prohibited to be used commercially or for personally, communally or privately unless permitted by author in writing.
- Copyrights 2023-2024 (c) are reserved only by the author: Najeeb Haider Zaidi
________________________________________________________________________
# Attributions:
The Dataset has been provided by Child Mind Institute. in [Kaggle Competition](https://www.kaggle.com/competitions/child-mind-institute-detect-sleep-states/overview) which the author is participating in and authorized to use the dataset solely for the competition purposes.
________________________________________________________________________

In [1]:
import numpy as np # linear algebra
import pandas as pd# data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime as dts
pd.set_option('display.max_row', 500)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/child-mind-institute-detect-sleep-states/train_series.parquet
/kaggle/input/child-mind-institute-detect-sleep-states/sample_submission.csv
/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv
/kaggle/input/child-mind-institute-detect-sleep-states/test_series.parquet


In [2]:
df=pd.read_parquet(path="/kaggle/input/child-mind-institute-detect-sleep-states/train_series.parquet", engine='auto')

In [3]:
train_events=pd.read_csv("/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv")

In [4]:
#'2018-08-14T15:30:00-0400'
#"2022-04-07T08:53:42.06717+02:00"
def tscv(dt):
    d=dts.strptime(dt, "%Y-%m-%dT%H:%M:%S%z")
    #d = dts.fromisoformat(dt)
    ts=dts.timestamp(d)
    #print('=', end ="")
    return ts

In [5]:
df['ts']=df['timestamp'].apply(lambda x: tscv(x))
df.sort_values(by=['series_id', 'ts','step'])
df.to_parquet('df_mod.parquet')


In [6]:
df.head()                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

,series_id,step,timestamp,anglez,enmo,ts
0,038441c925bb,0,2018-08-14T15:30:00-0400,2.6367,0.0217,1.534275e+09
1,038441c925bb,1,2018-08-14T15:30:05-0400,2.6368,0.0215,1.534275e+09
2,038441c925bb,2,2018-08-14T15:30:10-0400,2.6370,0.0216,1.534275e+09
3,038441c925bb,3,2018-08-14T15:30:15-0400,2.6368,0.0213,1.534275e+09
4,038441c925bb,4,2018-08-14T15:30:20-0400,2.6368,0.0215,1.534275e+09


In [7]:
df.sort_values(by=['series_id', 'ts','step'])

,series_id,step,timestamp,anglez,enmo,ts
0,038441c925bb,0,2018-08-14T15:30:00-0400,2.636700,0.0217,1.534275e+09
1,038441c925bb,1,2018-08-14T15:30:05-0400,2.636800,0.0215,1.534275e+09
2,038441c925bb,2,2018-08-14T15:30:10-0400,2.637000,0.0216,1.534275e+09
3,038441c925bb,3,2018-08-14T15:30:15-0400,2.636800,0.0213,1.534275e+09
4,038441c925bb,4,2018-08-14T15:30:20-0400,2.636800,0.0215,1.534275e+09
...,...,...,...,...,...,...
127946335,fe90110788d2,592375,2017-09-08T00:14:35-0400,-27.277500,0.0204,1.504844e+09
127946336,fe90110788d2,592376,2017-09-08T00:14:40-0400,-27.032499,0.0233,1.504844e+09
127946337,fe90110788d2,592377,2017-09-08T00:14:45-0400,-26.841200,0.0202,1.504844e+09
127946338,fe90110788d2,592378,2017-09-08T00:14:50-0400,-26.723900,0.0199,1.504844e+09
